<a href="https://colab.research.google.com/github/highway92/machine_learning/blob/main/year_dream/do/MLP%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B8%88%EC%9C%B5%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D(best_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix


import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
import torch
from torch import nn, optim

from torch.utils.data import DataLoader, Dataset

import torch.nn.functional as F

# 1. 데이터 로드 및 전처리
- feature selection
- MinMax Scaler

In [3]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')

In [4]:
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1]


In [5]:
sel = VarianceThreshold(threshold=0.01)
X = sel.fit_transform(X)


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
print('training set length :', len(X_train))
print('validation set length :', len(X_valid))

training set length : 80000
validation set length : 20000


In [6]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)

In [10]:
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]
X_test = sel.fit_transform(X_test)
X_test = scaler.fit_transform(X_test)

In [33]:
submission.head()

,ID,answer
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


# 2. MLP 구현 with pytorch

In [11]:
class TensorData(Dataset):
  def __init__(self, x_data, y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data.to_numpy())
    self.len = self.y_data.shape[0]

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

In [12]:
trainsets = TensorData(X_train, y_train)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size = 32, shuffle = True)

testsets = TensorData(X_valid, y_valid)
testloader = torch.utils.data.DataLoader(testsets, batch_size = 32, shuffle = False)

f1sets = TensorData(X_test, y_test)
f1loader = torch.utils.data.DataLoader(f1sets, batch_size = 32, shuffle = False)


In [13]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.dropout = nn.Dropout(p=0.2)
    self.linear1 = nn.Linear(63, 126)
    self.linear2 = nn.Linear(126, 48)
    self.linear3 = nn.Linear(48, 24)
    self.linear4 = nn.Linear(24, 2)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    mlp1 = F.relu(self.linear1(x))
    # print(mlp1.shape)
    mlp1 = self.dropout(mlp1)
    mlp2 = F.relu(self.linear2(mlp1))
    mlp2 = self.dropout(mlp2)
    mlp3 = F.relu(self.linear3(mlp2))
    mlp3 = self.dropout(mlp3)
    # print(mlp3.shape)
    mlp4 = F.relu(self.linear4(mlp3))
    # print(mlp4.shape)
    output = self.sigmoid(mlp4)
    return output


In [14]:
# model = MLP()
# input = torch.Tensor(1, 63)

# ot = model(input)
# print(ot)

In [15]:
# model = nn.Sequential(
#     nn.Linear(63, 126),
#     nn.ReLU(),
#     nn.Dropout(0.2),
#     nn.Linear(126, 48),
#     nn.ReLU(),
#     nn.Dropout(0.2),
#     nn.Linear(48, 24),
#     nn.ReLU(),
#     nn.Dropout(0.2),
#     nn.Linear(24, 2),
#     nn.Sigmoid()
# )

In [16]:
# X_trrain_tensor = torch.tensor(X_train, dtype = torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype = torch.int64)

In [47]:
EPOCHS = 200
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = MLP().to(DEVICE)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [48]:
from copy import deepcopy
loss_list = []
acc_list = []
best_acc = 0
best_f1 = 0
best_acc_model = None 
best_f1_model = None
from sklearn.metrics import f1_score


for epoch in range(EPOCHS):
    for i, (X_batch, y_batch) in enumerate(trainloader):
        #Forward 
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        y_output = model(X_batch)

        loss = criterion(y_output, y_batch) 

        #Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #misc (acc 계산, etc) 
        y_pred = torch.max(y_output, 1)[1]
        acc = accuracy_score(y_pred.data.cpu(), y_batch.data.cpu())
        f_score = f1_score(y_pred.data.cpu(), y_batch.data.cpu(), average='macro')
        if acc > best_acc:
          best_acc = acc
          best_acc_model = deepcopy(model.state_dict())

        if f_score > best_f1:
          best_f1 = f_score
          best_f1_model = deepcopy(model.state_dict())

        loss_list.append(loss.item())
        acc_list.append(acc)

    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}] Step [{}/{}] Loss: [{:.4f}] Train ACC [{:.2f}%] F1 Score [{:.2f}%]'.format(epoch+1, EPOCHS, \
                                                                                   i+1, len(trainloader), loss.item(), acc*100, f_score * 100))
        

Epoch [10/200] Step [2500/2500] Loss: [0.6400] Train ACC [62.50%] F1 Score [38.46%]
Epoch [20/200] Step [2500/2500] Loss: [0.6279] Train ACC [71.88%] F1 Score [41.82%]
Epoch [30/200] Step [2500/2500] Loss: [0.6158] Train ACC [68.75%] F1 Score [40.74%]
Epoch [40/200] Step [2500/2500] Loss: [0.6201] Train ACC [65.62%] F1 Score [39.62%]
Epoch [50/200] Step [2500/2500] Loss: [0.5944] Train ACC [78.12%] F1 Score [77.03%]
Epoch [60/200] Step [2500/2500] Loss: [0.5802] Train ACC [78.12%] F1 Score [66.21%]
Epoch [70/200] Step [2500/2500] Loss: [0.6503] Train ACC [59.38%] F1 Score [58.36%]
Epoch [80/200] Step [2500/2500] Loss: [0.6289] Train ACC [68.75%] F1 Score [61.35%]
Epoch [90/200] Step [2500/2500] Loss: [0.5525] Train ACC [81.25%] F1 Score [75.00%]
Epoch [100/200] Step [2500/2500] Loss: [0.5711] Train ACC [81.25%] F1 Score [72.57%]
Epoch [110/200] Step [2500/2500] Loss: [0.6077] Train ACC [71.88%] F1 Score [60.82%]
Epoch [120/200] Step [2500/2500] Loss: [0.5927] Train ACC [75.00%] F1 Scor

In [49]:
# best_acc_model

acc_model = MLP().to(DEVICE)
acc_model.load_state_dict(best_acc_model)
acc_model.eval()


MLP(
  (dropout): Dropout(p=0.2, inplace=False)
  (linear1): Linear(in_features=63, out_features=126, bias=True)
  (linear2): Linear(in_features=126, out_features=48, bias=True)
  (linear3): Linear(in_features=48, out_features=24, bias=True)
  (linear4): Linear(in_features=24, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [50]:
# best_acc_model

test_y_pred = []
test_acc_list = []
with torch.no_grad():

    for X_batch, y_batch in testloader:    
        #Forward
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        y_output = acc_model(X_batch)
        
        #misc (acc 계산, etc) 
        y_pred = torch.max(y_output, 1)[1]
        test_y_pred.extend(y_pred.data.cpu()) ##
        
        acc = accuracy_score(y_pred.data.cpu(), y_batch.data.cpu())
        test_acc_list.append(acc)
    test_acc = np.mean(test_acc_list)
print('Test ACC: [{:.2f}%]'.format(test_acc*100))
print('Test F1: [{:.2f}%]'.format(f1_score(list(y_valid),test_y_pred, average='macro')*100))

Test ACC: [71.80%]
Test F1: [59.41%]


In [51]:
# best_f1_score_model
f1_model = MLP().to(DEVICE)
f1_model.load_state_dict(best_f1_model)
f1_model.eval()

MLP(
  (dropout): Dropout(p=0.2, inplace=False)
  (linear1): Linear(in_features=63, out_features=126, bias=True)
  (linear2): Linear(in_features=126, out_features=48, bias=True)
  (linear3): Linear(in_features=48, out_features=24, bias=True)
  (linear4): Linear(in_features=24, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [52]:
# best_f1_score_model

test_y_pred = []
test_acc_list = []
with torch.no_grad():

    for X_batch, y_batch in testloader:    
        #Forward
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        y_output = f1_model(X_batch)
        
        #misc (acc 계산, etc) 
        y_pred = torch.max(y_output, 1)[1]
        test_y_pred.extend(y_pred.data.cpu()) ##
        
        acc = accuracy_score(y_pred.data.cpu(), y_batch.data.cpu())
        test_acc_list.append(acc)
    test_acc = np.mean(test_acc_list)
print('Test ACC: [{:.2f}%]'.format(test_acc*100))
print('Test F1: [{:.2f}%]'.format(f1_score(list(y_valid),test_y_pred, average='macro')*100))


Test ACC: [71.80%]
Test F1: [59.41%]


In [53]:
# prediction

test_y_pred = []
test_acc_list = []
with torch.no_grad():

    for X_batch, y_batch in f1loader:    
        #Forward
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        y_output = f1_model(X_batch)
        
        #misc (acc 계산, etc) 
        y_pred = torch.max(y_output, 1)[1]
        test_y_pred.extend(y_pred.data.cpu()) ##
        
        acc = accuracy_score(y_pred.data.cpu(), y_batch.data.cpu())
        test_acc_list.append(acc)

In [54]:
test_y_pred = list(map(int, test_y_pred))

In [55]:
submission['answer'] = test_y_pred
submission.to_csv('./prediction.csv', index = False)